In [1]:
# csv 导入数据
def load_csvdata(csvFileName):
    import pandas as pd
    result = pd.read_csv(open(csvFileName))
    return result

In [11]:
train_full_raw = load_csvdata("train_full.csv")
train_count_raw = load_csvdata("train_count.csv")
train_tfidf_raw = load_csvdata("train_tfidf.csv")
train_glove300_raw = load_csvdata("train_glove300.csv")

dev_full_raw = load_csvdata("dev_full.csv")
dev_count_raw = load_csvdata("dev_count.csv")
dev_tfidf_raw = load_csvdata("dev_tfidf.csv")
dev_glove300_raw = load_csvdata("dev_glove300.csv")

test_full_raw = load_csvdata("test_full.csv")
test_count_raw = load_csvdata("test_count.csv")
test_tfidf_raw = load_csvdata("test_tfidf.csv")
test_glove300_raw = load_csvdata("test_glove300.csv")

In [15]:
train_label = train_full_raw["region"]
dev_label = dev_full_raw["region"]

In [19]:
train_full = train_full_raw["tweet"]
train_count = train_count_raw["tweet"]
train_tfidf = train_tfidf_raw["tweet"]
train_glove300 = train_glove300_raw["tweet"]

dev_full = dev_full_raw["tweet"]
dev_count = dev_count_raw["tweet"]
dev_tfidf = dev_tfidf_raw["tweet"]
dev_glove300 = dev_glove300_raw["tweet"]

test_full = test_full_raw["tweet"]
test_count = test_count_raw["tweet"]
test_tfidf = test_tfidf_raw["tweet"]
test_glove300 = test_glove300_raw["tweet"]

In [29]:
vocab = open("vocab.txt", "r")
vocab_dic = {}
for i in vocab.readlines():
    value, key = i.replace("\n", "").split("\t") #re
    vocab_dic[key] = value
# lines = vocab.readlines()
# value, key = lines[0].replace("\n", "").split("\t")
# value
# key

In [49]:
# r : 后面不需要加 \ 就可以表示特殊字符
# ‘’: 匹配规则
# [(] : [] 里面的只是一个平凡的(
# (.*?): ()是规则表达式
# . :!\n
# * 克林闭包 0 到 +∞循环
# ？最小匹配， 没有？叫贪婪匹配
# import re
# # print(re.findall(r'[(](.*)[)]', train_count[0]))
# one = re.findall(r'[(](.*?)[)]', train_count[0])
# #oneLine = [0] * len(vocab_dic)
# oneLine = [0 for i in range(len(vocab_dic))]
# temp = one[0].split(",")
# temp
# oneLine[int(temp[0])] = float(temp[1])
# oneLine[1111]


'1111, 1.0'

In [52]:
import re
def get_train_vector(train):
    result = []
    for i in train:
        temp = re.findall(r'[(](.*?)[)]', i)
        oneLine = [0 for i in range(len(vocab_dic))]
        for j in temp:
            index, value = j.split(",")
            oneLine[int(index)] = float(value)
        result.append(oneLine)
    return result

In [60]:
train_count_vector = get_train_vector(train_count)
train_tfidf_vector = get_train_vector(train_tfidf)

dev_count_vector = get_train_vector(dev_count)
dev_tfidf_vector = get_train_vector(dev_tfidf)

test_count_vector = get_train_vector(test_count)
test_tfidf_vector = get_train_vector(test_tfidf)

In [61]:
import pandas as pd
train_count_frame = pd.DataFrame(train_count_vector)
train_tfidf_frame = pd.DataFrame(train_tfidf_vector)

dev_count_frame = pd.DataFrame(dev_count_vector)
dev_tfidf_frame = pd.DataFrame(dev_tfidf_vector)

test_count_frame = pd.DataFrame(test_count_vector)
test_tfidf_frame = pd.DataFrame(test_tfidf_vector)

"""
有一列全部都是0， del
"""

In [62]:
# def KNN(trainX, trainY, k):
#     from sklearn.neighbors import KNeighborsClassifier
#     knn = KNeighborsClassifier(n_neighbors = k)
#     knn.fit(trainX, trainY)
#     return knn

def MNB(trainX, trainY):
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(trainX, trainY)
    return mnb

def BNB(trainX, trainY):
    from sklearn.naive_bayes import BernoulliNB
    bnb = BernoulliNB()
    bnb.fit(trainX, trainY)
    return bnb

def GNB(trainX, trainY):
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    gnb.fit(trainX, trainY)
    return gnb

# def Detree(trainX, trainY):
#     from sklearn.tree import DecisionTreeClassifier
#     Dtree = DecisionTreeClassifier(criterion='gini') # 
#     Dtree.fit(trainX, trainY)
#     return Dtree

# def RandomF(trainX, trainY):
#     from sklearn.ensemble import RandomForestClassifier
#     Rfc = RandomForestClassifier(criterion='entropy')
#     Rfc.fit(train_X, train_Y)
#     return Rfc

def predict(model, test):
    return model.predict(test)

def evaluate(Y_real, Y_predict):
    from sklearn.metrics import accuracy_score
    return accuracy_score(Y_real, Y_predict)

In [63]:
mnb_count = MNB(train_count_vector, train_label)
mnb_tfidf = MNB(train_tfidf_vector, train_label)

bnb_count = BNB(train_count_vector, train_label)
bnb_tfidf = BNB(train_tfidf_vector, train_label)

gnb_count = GNB(train_count_vector, train_label)
gnb_tfidf = GNB(train_tfidf_vector, train_label)

In [64]:
mnb_count_predict = predict(mnb_count, dev_count_vector)
mnb_tfidf_predict = predict(mnb_tfidf, dev_tfidf_vector)

bnb_count_predict = predict(bnb_count, dev_count_vector)
bnb_tfidf_predict = predict(bnb_tfidf, dev_tfidf_vector)

gnb_count_predict = predict(gnb_count, dev_count_vector)
gnb_tfidf_predict = predict(gnb_tfidf, dev_tfidf_vector)

In [65]:
print("MNB_count acc :", evaluate(dev_label, mnb_count_predict))
print("MNB_tfidf acc :", evaluate(dev_label, mnb_tfidf_predict))

print("BNB_count acc :", evaluate(dev_label, bnb_count_predict))
print("BNB_tfidf acc :", evaluate(dev_label, bnb_tfidf_predict))

print("GNB_count acc :", evaluate(dev_label, gnb_count_predict))
print("GNB_tfidf acc :", evaluate(dev_label, gnb_tfidf_predict))

MNB_count acc : 0.4541176470588235
MNB_tfidf acc : 0.46352941176470586
BNB_count acc : 0.45647058823529413
BNB_tfidf acc : 0.45647058823529413
GNB_count acc : 0.22492374727668846
GNB_tfidf acc : 0.23093681917211328


In [77]:
def submit(model, test):
    predict_test = predict(model, test)
    myId = [i for i in range(1, len(predict_test) + 1)]
    result = pd.concat([pd.DataFrame(myId), pd.DataFrame(predict_test)], axis = 1)
    result.columns = ["id", "region"]
    result.to_csv(r"courseDemo.csv", index = False, header = True)

In [78]:
submit(bnb_tfidf, test_tfidf_vector)

In [87]:
def get_train_vector_glove300(glove300):
    result = []
    for i in glove300:
        temp = i.split(" ")
        oneLine = []
        for j in temp:
            oneLine.append(float(j))
        result.append(oneLine)
    return result

In [95]:
train_count_frame.columns = list(vocab_dic.values())

In [96]:
train_count_frame

,!,"""",#,#amitheonlyone,#andthenwehadsex,#bears,#beforeidie,#betmessedupwhen,#bgc,#chancesare,...,¥,¦,©,«,°,¹,º,»,â,î
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133791,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133792,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
133793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
from collections import Counter
Counter(train_label)
O_R = ['NORTHEAST' for i in range(len(dev_label))]
len(O_R) == len(dev_label)

True